# Functions

## General Functions
* Used throughout the notebook

In [1]:
# Check number of items in the Resources subdirectories
# Purpose: Automate file naming
def recipe_folder():
    count = len(os.listdir('Resources/01_recipe_IDs'))-1
    return(count)

def raw_folder():
    count = len(os.listdir('Resources/02_raw_data'))-1
    return(count)

def simple_folder():
    count = len(os.listdir('Resources/03_simplified_data'))-1
    return(count)

def complex_folder():
    count = len(os.listdir('Resources/04_complex_test'))-1
    return(count)

In [2]:
# Purpose: Check each value in a list whether it exists in the tags
def tag_check(input_list):
    """
    The purpose of this function is to check whether each value in list
    exists in a longer list (i.e. `tags` column in the Food.com dataset):

    Args:
    input_list (:obj: `list` of :obj:`str`): List of values to search for

    Returns:
    output_list (:obj: `list` of :obj:`int`): 
        output_list[0] (`int`): Values that exist in the longer list
        output_list[1] (`int`): Values that do not exist in the longer list
        output_list[2] (`int`): Alternatives to values that do not exist
    """
    yes_tag = []
    not_tag = []
    alt_tag = []

    # Loop through each tag in the list
    for tag in input_list:
        if tag in unique_tags:
            yes_tag.append(tag)
        else:
            not_tag.append(tag)
            # Check for alternative tags
            for values in unique_tags:
                if tag in values:
                    alt_tag.append(values)
    output_list = [yes_tag, not_tag, alt_tag]
    return(output_list)

In [3]:
# Purpose: Get a count of a given list from the tags column, and return its id
def parse_tags(input_list, df, save_col):
    # Create a dictionary to hold the count and list of IDs
    word_dict = {}
    for word in input_list:
        word_dict[word] = dict(count = 0, id_list = [])

    # Add a `multiple` key to track mutual exclusivity
    word_dict['multiple'] = dict(count = 0, id_list = [])

    for df_idx, string in df.iterrows():
        tag_list = string['tags'].strip("[]").split(', ')
    
        # Track mutual exclusivity for the meal types
        score = 0
        for word_idx in range(len(tag_list)):
            # Check the tag
            tag = tag_list[word_idx].strip("'")
            if tag in input_list:
                word_dict[tag]['count'] += 1
                word_dict[tag]['id_list'].append(string[save_col])
                score += 1
                # If >1 tag in a list, add to 'multiple' and continue
                if (score > 1):
                    word_dict['multiple']['count'] += 1
                    word_dict['multiple']['id_list'].append(string[save_col])
                    continue
    return(word_dict)

In [4]:
# Define a function to create linear regression plots
def linreg_plot(ax_object, x_values, y_values, x_coord, y_coord):
    # Calculate the lienar regression for x- and y-values
    (slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

    # Get regression values and equation
    regression_values = x_values * slope + intercept
    line_equation = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

    # Calculate the r-value
    print(f'The r-value is: {rvalue**2}')

    # Plot the linear regression
    ax_object.plot(x_values, regression_values, color='red')
    ax_object.annotate(
        line_equation, xy=(x_coord, y_coord), xycoords='figure points',
        fontsize = 15, color='red', weight='bold')
    plt.show()

In [5]:
# Parse 'nutrition' column and return nutritional values
def get_nutrition(string, nutritional_val):
    result = []
    # Based on longest name: Mono Unsaturated Fat
    longest_length = 90
    
    # Get the index of where the substring starts
    start_index = string.index(f"'{nutritional_val}")

    # Get the entire section of the nutritional value details
    section = string[start_index : start_index + longest_length]

    # Find the first closing curly bracket, set as the end_index
    curly_bracket = section.find("}") # returns -1 if not found
    
    if (curly_bracket > 0):
        end_index = start_index + curly_bracket
        section = string[start_index : end_index]

    section_list = section.split(",")
    for idx, part in enumerate(section_list):
        parsed_list = part.split(":")
        # Get the amount
        if (idx == 1):
            result.append(float(parsed_list[1].strip(" '")))
        if (idx == 2):
            result.append((parsed_list[1].strip(" '")))
        if (idx == 3):
            result.append(float(parsed_list[1].strip(" '")))
    return(result)

In [6]:
# Get the IQR, and lower and upper bounds, and append to DataFrame
def add_iqr(input_stats):
    # Check if DataFrame or Series
    if (len(input_stats.shape) == 1):
        # Code for Series as an input
        output_index = list(input_stats.index) + ['iqr', 'lower_bounds', 'upper_bounds']
        output_value = list(input_stats.values)

        # Get inputs
        lower = input_stats['25%']
        upper = input_stats['75%']
        iqr = upper - lower

        # Append to output list
        output_value.append(iqr)
        output_value.append(lower - 1.5*iqr)
        output_value.append(upper + 1.5*iqr)

        output_series = pd.Series(output_value, index=output_index)
        return(output_series)

    elif (len(input_stats.shape) > 1):
        iqr_list = []
        lowerbounds_list = []
        upperbounds_list = []
        above_max = []
        for df_idx, row in input_stats.iterrows():
            lower = input_stats.loc[df_idx, '25%']
            upper = input_stats.loc[df_idx, '75%']
            iqr = upper - lower
            
            iqr_list.append(iqr)
            lowerbounds_list.append(lower - 1.5*iqr)
            upperbounds_list.append(upper + 1.5*iqr)
        
            if (input_stats.loc[df_idx, 'max'] > upper):
                above_max.append(df_idx)
        
        # Add results to the descriptive statistics table
        input_stats['iqr'] = iqr_list
        input_stats['lower_bounds'] = lowerbounds_list
        input_stats['upper_bounds'] = upperbounds_list
        
        return(above_max)

## Spoonacular API Requests

## Spoonacular API Data Collection - Method 1
- API request cost comparison:
    * Type 1: `Random Recipes` + `Recipe Information` = 2.1 points per request
    * Type 2: `Random Recipes` + `Nutrition by ID` = 2 points per request

In [7]:
def spoonacular_v1(type):
    ### PHASE 1 - Get Recipe IDs ###
    # API REQUEST - Random Recipe
    random_url = "https://api.spoonacular.com/recipes/random?"

    # Set request parameters
    random_params = {
        'apiKey': spoonacular_key,
        'number': 100}
    
    # Request a list of random recipe IDs.
    random_response = requests.get(random_url, random_params).json()
    
    # Output response to a csv
    random_df = pd.DataFrame(random_response['recipes'])
    random_df.to_csv(f'Resources/01_recipe_IDs/initial_recipes_{recipe_folder()}.csv', index=False)

    ### PHASE 2 - Get Nutrition Information (2 Types) ###
    if (type == 1):
        # API REQUEST - Recipe Information
        nutrition_info = []
        for id in random_df['id']:
            nutrition_url = f"https://api.spoonacular.com/recipes/{id}/information?"
            nutrition_params = {
                'apiKey': spoonacular_key,
                'includeNutrition': "true"}
            
            # Request recipe information for each recipe ID
            nutrition_response = requests.get(nutrition_url, nutrition_params).json()
            nutrition_info.append(nutrition_response)
    
    if (type == 2):
        # API REQUEST - Nutrition by ID
        nutrition_info = []
        for id in random_df['id']:
            nutrition_url = f'https://api.spoonacular.com/recipes/{id}/nutritionWidget.json'
            nutrition_params = {'apiKey': spoonacular_key}
    
            # Request nutrition information for each recipe ID
            nutrition_response = requests.get(nutrition_url, nutrition_params).json()
            nutrition_info.append(nutrition_response)

    # Output response to a csv
    nutrition_df = pd.DataFrame(nutrition_info)
    nutrition_df.to_csv(f'Resources/02_raw_data/info_master_{raw_folder()}.csv')

    ### PHASE 3 - Parse response ###
    # Metadata
    metadata = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_{recipe_folder()-1}.csv')
    num_ingredients = []
    carbs = []

    # Calories
    calories_amt = []
    calories_unit = []
    calories_pct = []
    
    # Saturated Fat
    satfat_amt = []
    satfat_unit = []
    satfat_pct = []
    
    # Sugar
    sugar_amt = []
    sugar_unit = []
    sugar_pct = []
    
    # Sodium
    sodium_amt = []
    sodium_unit = []
    sodium_pct = []
    
    # Protein
    protein_amt = []
    protein_unit = []
    protein_pct = []
    
    for idx, row in enumerate(nutrition_info):
        try:
            num_ingredients.append(len(nutrition_info[idx]['ingredients']))
            carbs.append(nutrition_info[idx]['carbs'])
            calories_amt.append(nutrition_info[idx]['nutrients'][0]['amount'])
            calories_unit.append(nutrition_info[idx]['nutrients'][0]['unit'])
            calories_pct.append(nutrition_info[idx]['nutrients'][0]['percentOfDailyNeeds'])
            
            satfat_amt.append(nutrition_info[idx]['nutrients'][2]['amount'])
            satfat_unit.append(nutrition_info[idx]['nutrients'][2]['unit'])
            satfat_pct.append(nutrition_info[idx]['nutrients'][2]['percentOfDailyNeeds'])
            
            sugar_amt.append(nutrition_info[idx]['nutrients'][5]['amount'])
            sugar_unit.append(nutrition_info[idx]['nutrients'][5]['unit'])
            sugar_pct.append(nutrition_info[idx]['nutrients'][5]['percentOfDailyNeeds'])
            
            sodium_amt.append(nutrition_info[idx]['nutrients'][7]['amount'])
            sodium_unit.append(nutrition_info[idx]['nutrients'][7]['unit'])
            sodium_pct.append(nutrition_info[idx]['nutrients'][7]['percentOfDailyNeeds'])
            
            protein_amt.append(nutrition_info[idx]['nutrients'][8]['amount'])
            protein_unit.append(nutrition_info[idx]['nutrients'][8]['unit'])
            protein_pct.append(nutrition_info[idx]['nutrients'][8]['percentOfDailyNeeds'])
        
        except:
            print(idx)
    
    spoonacular_df = pd.DataFrame({
        'ID': metadata['id'],
        'Name': metadata['title'],
        'Likes': metadata['aggregateLikes'],
        'Meal Type': metadata['dishTypes'],
        'Cuisines': metadata['cuisines'],
        'N_ingredients': num_ingredients,
        'WW Smart Points': metadata['weightWatcherSmartPoints'],
        'Calories (Amount)': calories_amt,
        'Calories (Unit)': calories_unit,
        'Calories (% of Daily Needs)': calories_pct,
        'Saturated Fat (Amount)': satfat_amt,
        'Saturated Fat (Unit)': satfat_unit,
        'Saturated Fat (% of Daily Needs)': satfat_pct,
        'Sugar (Amount)': sugar_amt,
        'Sugar (Unit)': sugar_unit,
        'Sugar (% of Daily Needs)': sugar_pct,
        'Sodium (Amount)': sodium_amt,
        'Sodium (Unit)': sodium_unit,
        'Sodium (% of Daily Needs)': sodium_pct,
        'Protein (Amount)': protein_amt,
        'Protein (Unit)': protein_unit,
        'Protein (% of Daily Needs)': protein_pct,
        'Carbohydrates': carbs})
    
    spoonacular_df.to_csv(f'Resources/03_simplified_data/initial_nutrition_{simple_folder()}.csv', index=False)

## Spoonacular API Data Collection - Method 2
* Identified as the most cost-efficient method, a single request with all the details.

In [8]:
def spoonacular_v2():
    ### PHASE 1 - Get Recipes with all information required ###
    # API REQUEST - Complex Search.
    recipe_url = "https://api.spoonacular.com/recipes/complexSearch"
    
    # Set request parameters
    recipe_params = {
        'apiKey': spoonacular_key,
        'number': 100,
        'cuisine': cuisine_match,
        'type': "breakfast", # Run again with "lunch" and "dinner"
        'addRecipeNutrition': True}
    
    # Request a list of recipes.
    recipe_response = requests.get(recipe_url, recipe_params).json()
    
    # Output response to a csv
    recipes_df = pd.DataFrame(recipe_response['results'])
    recipes_df.to_csv(f'Resources/02_raw_data/recipe_data_{raw_folder()}.csv', index=False)

    ### PHASE 2 - Parse response ###
    # Metadata
    recipe_id = []
    recipe_name = []
    recipe_likes = []
    meal_type = []
    cuisines = []
    num_ingredients = []
    ww_points = []
    mins_cook = []
    mins_prep = []
    
    # Calories
    calories_amt = []
    calories_unit = []
    calories_pct = []
    
    # Saturated Fat
    satfat_amt = []
    satfat_unit = []
    satfat_pct = []
    
    # Sugar
    sugar_amt = []
    sugar_unit = []
    sugar_pct = []
    
    # Sodium
    sodium_amt = []
    sodium_unit = []
    sodium_pct = []
    
    # Protein
    protein_amt = []
    protein_unit = []
    protein_pct = []
    
    # Carbohydrates
    carbs_amt = []
    carbs_unit = []
    carbs_pct = []
    
    recipe_info = recipe_response['results']
    for idx, row in enumerate(recipe_info):
        recipe_id.append(recipe_info[idx]['id'])
        recipe_name.append(recipe_info[idx]['title'])
        recipe_likes.append(recipe_info[idx]['aggregateLikes'])
        meal_type.append(recipe_info[idx]['dishTypes'])
        cuisines.append(recipe_info[idx]['cuisines'])
        num_ingredients.append(len(recipe_info[idx]['nutrition']['ingredients']))
        ww_points.append(recipe_info[idx]['weightWatcherSmartPoints'])
        mins_cook.append(recipe_info[idx]['cookingMinutes'])
        mins_prep.append(recipe_info[idx]['preparationMinutes'])
        
        calories_amt.append(recipe_info[idx]['nutrition']['nutrients'][0]['amount'])
        calories_unit.append(recipe_info[idx]['nutrition']['nutrients'][0]['unit'])
        calories_pct.append(recipe_info[idx]['nutrition']['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(recipe_info[idx]['nutrition']['nutrients'][2]['amount'])
        satfat_unit.append(recipe_info[idx]['nutrition']['nutrients'][2]['unit'])
        satfat_pct.append(recipe_info[idx]['nutrition']['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(recipe_info[idx]['nutrition']['nutrients'][5]['amount'])
        sugar_unit.append(recipe_info[idx]['nutrition']['nutrients'][5]['unit'])
        sugar_pct.append(recipe_info[idx]['nutrition']['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(recipe_info[idx]['nutrition']['nutrients'][7]['amount'])
        sodium_unit.append(recipe_info[idx]['nutrition']['nutrients'][7]['unit'])
        sodium_pct.append(recipe_info[idx]['nutrition']['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(recipe_info[idx]['nutrition']['nutrients'][8]['amount'])
        protein_unit.append(recipe_info[idx]['nutrition']['nutrients'][8]['unit'])
        protein_pct.append(recipe_info[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
    
        carbs_amt.append(recipe_info[idx]['nutrition']['nutrients'][8]['amount'])
        carbs_unit.append(recipe_info[idx]['nutrition']['nutrients'][8]['unit'])
        carbs_pct.append(recipe_info[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
    
    spoonacular_df = pd.DataFrame({
        'ID': recipe_id,
        'Name': recipe_name,
        'Likes': recipe_likes,
        'Meal Type': meal_type,
        'Cuisines': cuisines,
        'N_ingredients': num_ingredients,
        'WW Smart Points': ww_points,
        'Cooking Minutes': mins_cook,
        'Prep Minutes': mins_prep,
        'Calories (Amount)': calories_amt,
        'Calories (Unit)': calories_unit,
        'Calories (% of Daily Needs)': calories_pct,
        'Saturated Fat (Amount)': satfat_amt,
        'Saturated Fat (Unit)': satfat_unit,
        'Saturated Fat (% of Daily Needs)': satfat_pct,
        'Sugar (Amount)': sugar_amt,
        'Sugar (Unit)': sugar_unit,
        'Sugar (% of Daily Needs)': sugar_pct,
        'Sodium (Amount)': sodium_amt,
        'Sodium (Unit)': sodium_unit,
        'Sodium (% of Daily Needs)': sodium_pct,
        'Protein (Amount)': protein_amt,
        'Protein (Unit)': protein_unit,
        'Protein (% of Daily Needs)': protein_pct,
        'Carbs (Amount)': carbs_amt,
        'Carbs (Unit)': carbs_unit,
        'Carbs (% of Daily Needs)': carbs_pct})
    
    spoonacular_df.to_csv(f'Resources/04_complex_test/simplified_data_{complex_folder()}.csv', index=False)

## Spoonacular API Data Collection - Method 3
* Used for getting information when Recipe IDs already exist.

In [10]:
def spoonacular_v3(recipe_ids):
    ### PHASE 1 - Get Recipes with all information required ###
    # API REQUEST - Recipe Information
    nutrition_info = []
    for id in missing_ids:
        nutrition_url = f"https://api.spoonacular.com/recipes/{id}/information?"
        nutrition_params = {
            'apiKey': spoonacular_key,
            'includeNutrition': "true"}
        
        # Request recipe information for each recipe ID
        nutrition_response = requests.get(nutrition_url, nutrition_params).json()
        nutrition_info.append(nutrition_response)

    # Output response to a csv
    recipes_df = pd.DataFrame(nutrition_info)
    recipes_df.to_csv(f'Resources/02_raw_data/recipe_data_{raw_folder()}.csv', index=False)

    ### PHASE 2 - Parse response ###
    # Metadata
    recipe_id = []
    recipe_name = []
    recipe_likes = []
    meal_type = []
    cuisines = []
    num_ingredients = []
    ww_points = []
    mins_cook = []
    mins_prep = []
    
    # Calories
    calories_amt = []
    calories_unit = []
    calories_pct = []
    
    # Saturated Fat
    satfat_amt = []
    satfat_unit = []
    satfat_pct = []
    
    # Sugar
    sugar_amt = []
    sugar_unit = []
    sugar_pct = []
    
    # Sodium
    sodium_amt = []
    sodium_unit = []
    sodium_pct = []
    
    # Protein
    protein_amt = []
    protein_unit = []
    protein_pct = []
    
    # Carbohydrates
    carbs_amt = []
    carbs_unit = []
    carbs_pct = []
    
    recipe_info = nutrition_info
    for idx, row in enumerate(recipe_info):
        try:
            recipe_id.append(recipe_info[idx]['id'])
            recipe_name.append(recipe_info[idx]['title'])
            recipe_likes.append(recipe_info[idx]['aggregateLikes'])
            meal_type.append(recipe_info[idx]['dishTypes'])
            cuisines.append(recipe_info[idx]['cuisines'])
            num_ingredients.append(len(recipe_info[idx]['nutrition']['ingredients']))
            ww_points.append(recipe_info[idx]['weightWatcherSmartPoints'])
            mins_cook.append(recipe_info[idx]['cookingMinutes'])
            mins_prep.append(recipe_info[idx]['preparationMinutes'])
            
            calories_amt.append(recipe_info[idx]['nutrition']['nutrients'][0]['amount'])
            calories_unit.append(recipe_info[idx]['nutrition']['nutrients'][0]['unit'])
            calories_pct.append(recipe_info[idx]['nutrition']['nutrients'][0]['percentOfDailyNeeds'])
            
            satfat_amt.append(recipe_info[idx]['nutrition']['nutrients'][2]['amount'])
            satfat_unit.append(recipe_info[idx]['nutrition']['nutrients'][2]['unit'])
            satfat_pct.append(recipe_info[idx]['nutrition']['nutrients'][2]['percentOfDailyNeeds'])
            
            sugar_amt.append(recipe_info[idx]['nutrition']['nutrients'][5]['amount'])
            sugar_unit.append(recipe_info[idx]['nutrition']['nutrients'][5]['unit'])
            sugar_pct.append(recipe_info[idx]['nutrition']['nutrients'][5]['percentOfDailyNeeds'])
            
            sodium_amt.append(recipe_info[idx]['nutrition']['nutrients'][7]['amount'])
            sodium_unit.append(recipe_info[idx]['nutrition']['nutrients'][7]['unit'])
            sodium_pct.append(recipe_info[idx]['nutrition']['nutrients'][7]['percentOfDailyNeeds'])
            
            protein_amt.append(recipe_info[idx]['nutrition']['nutrients'][8]['amount'])
            protein_unit.append(recipe_info[idx]['nutrition']['nutrients'][8]['unit'])
            protein_pct.append(recipe_info[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
        
            carbs_amt.append(recipe_info[idx]['nutrition']['nutrients'][8]['amount'])
            carbs_unit.append(recipe_info[idx]['nutrition']['nutrients'][8]['unit'])
            carbs_pct.append(recipe_info[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
        except:
            continue
    
    spoonacular_df = pd.DataFrame({
        'ID': recipe_id,
        'Name': recipe_name,
        'Likes': recipe_likes,
        'Meal Type': meal_type,
        'Cuisines': cuisines,
        'N_ingredients': num_ingredients,
        'WW Smart Points': ww_points,
        'Cooking Minutes': mins_cook,
        'Prep Minutes': mins_prep,
        'Calories (Amount)': calories_amt,
        'Calories (Unit)': calories_unit,
        'Calories (% of Daily Needs)': calories_pct,
        'Saturated Fat (Amount)': satfat_amt,
        'Saturated Fat (Unit)': satfat_unit,
        'Saturated Fat (% of Daily Needs)': satfat_pct,
        'Sugar (Amount)': sugar_amt,
        'Sugar (Unit)': sugar_unit,
        'Sugar (% of Daily Needs)': sugar_pct,
        'Sodium (Amount)': sodium_amt,
        'Sodium (Unit)': sodium_unit,
        'Sodium (% of Daily Needs)': sodium_pct,
        'Protein (Amount)': protein_amt,
        'Protein (Unit)': protein_unit,
        'Protein (% of Daily Needs)': protein_pct,
        'Carbs (Amount)': carbs_amt,
        'Carbs (Unit)': carbs_unit,
        'Carbs (% of Daily Needs)': carbs_pct})
    
    spoonacular_df.to_csv(f'Resources/04_complex_test/simplified_data_{complex_folder()}.csv', index=False)